### Problem 1a

In [ ]:
## Setup
# load packages

# load data


## Estimate coefficients (VVV functionalize this VVV)
# separate into features vs. outputs

# add intercept term to features

# solve for (X^T*X)^-1

# beta_est = [(X^T*X)^-1] * X^T * Y


## check work (??)

Problem 1b

In [ ]:
## Split into training/validation sets (75/25)

## Build alternative models (first 7/8/9 columns)

## Fit models to training data 

## Compute MSE with test data